## Import the libraries we need

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from splinter import Browser
from pprint import pprint
from datetime import datetime
import pymongo
from bson.json_util import loads
import json 
import pytz

# Setting upt our Mongo DB

In [3]:
print("Initialize PyMongo to work with MongoDBs")
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.travel_informers
collection = db.city_pics
print('Mongo Initialized')

Initialize PyMongo to work with MongoDBs
Mongo Initialized


# Initialize the Chromedriver

In [37]:
executable_path = {'executable_path':'D:/Chromedriver/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

# get our list of cities
** And format the city names so that they match the url formats of wikipedia **  
 for example: https://en.wikipedia.org/wiki/San_Francisco  
 note the underscore

In [38]:
import pandas as pd
df = pd.read_csv('historical-hourly-weather-data/city_attributes.csv')

city_list = list(df.City[:30])
#print(city_list)

city_list = ['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles', 'San Diego', 'Las Vegas', 
             'Phoenix', 'Albuquerque', 'Denver', 'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis',
             'St._Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte',
             'Miami', 'Pittsburgh', 'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston']

# add underscors to city names that need them
clean_city_list = []
for city in city_list:
    if ' ' in city:
        clean_city_list.append(city.replace(' ', '_'))
    else:
        clean_city_list.append(city)

print("city names prepared for wikipedia.org: \n",clean_city_list)



city names prepared for wikipedia.org: 
 ['Vancouver', 'Portland', 'San_Francisco', 'Seattle', 'Los_Angeles', 'San_Diego', 'Las_Vegas', 'Phoenix', 'Albuquerque', 'Denver', 'San_Antonio', 'Dallas', 'Houston', 'Kansas_City', 'Minneapolis', 'St._Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta', 'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh', 'Toronto', 'Philadelphia', 'New_York', 'Montreal', 'Boston']


In [48]:
base_url = "https://en.wikipedia.org/wiki/"

executable_path = {'executable_path':'D:/Chromedriver/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

# clear out collection before scraping
# collection.delete_many({})
city_urls = {}
for city in clean_city_list:
    if collection.find_one({'city': city}):
        print(collection.find_one({'city':city}))
    else:
        record = {}
        record['city'] = city
        city_url = base_url + city
        browser.visit(city_url)
        soup = bs(browser.html, 'html.parser')

        #for title in soup.find_all("table", class_='infobox geography vcard'):
             #print(title.find_all('tr'))

        pics = []
        for pic in soup.find_all('a', class_='image'):
            img_url = 'https://en.wikipedia.org' + pic['href']
            print(img_url)
            browser.visit(img_url)
            pic_soup = bs(browser.html, 'html.parser')
            for element in pic_soup.find_all('div', class_='fullImageLink'):
                img_src = element.find('a').find('img')['src']
                pics.append(img_src)


        record['images'] = pics

        print(record)
        collection.insert_one(record)


https://en.wikipedia.org/wiki/File:Downtown_Vancouver_2017.jpg
https://en.wikipedia.org/wiki/File:Flag_of_Vancouver_(Canada).svg
https://en.wikipedia.org/wiki/File:Vancouverlogo.svg
https://en.wikipedia.org/wiki/File:GVRDVancouver.svg
https://en.wikipedia.org/wiki/File:Canada_British_Columbia_relief_location_map.jpg
https://en.wikipedia.org/wiki/File:Relief_map_of_Canada.png
https://en.wikipedia.org/wiki/File:North_America_laea_relief_location_map.jpg
https://en.wikipedia.org/wiki/File:Maple_Tree_Corner_Vancouver_1886.jpg
https://en.wikipedia.org/wiki/File:First_Vancouver_Council_Meeting_after_fire.jpg
https://en.wikipedia.org/wiki/File:RCMP_1938_sitdowner_strike.jpg
https://en.wikipedia.org/wiki/File:Stadtgliederung_Vancouver_2008.png
https://en.wikipedia.org/wiki/File:Stanley_Park_1999_Rain.jpg
https://en.wikipedia.org/wiki/File:Vancouver_aerial_view.jpg
https://en.wikipedia.org/wiki/File:Vancouver_dusk_pano.jpg
https://en.wikipedia.org/wiki/File:Robson_Square_Vancouver_02.JPG
https:

### For individual cities that didn't scrape so well`

In [11]:
executable_path = {'executable_path':'D:/Chromedriver/chromedriver.exe'}
browser = Browser('chrome', **executable_path)
browser.visit("https://en.wikipedia.org/wiki/New_York_City")
soup = bs(browser.html, 'html.parser')

        #for title in soup.find_all("table", class_='infobox geography vcard'):
             #print(title.find_all('tr'))

pics = []
for pic in soup.find_all('a', class_='image'):
    img_url = 'https://en.wikipedia.org' + pic['href']
    print(img_url)
    browser.visit(img_url)
    pic_soup = bs(browser.html, 'html.parser')
    for element in pic_soup.find_all('div', class_='fullImageLink'):
        img_src = element.find('a').find('img')['src']
        pics.append(img_src)
print(pics)

https://en.wikipedia.org/wiki/File:NYC_Montage_2014_4_-_Jleon.jpg
https://en.wikipedia.org/wiki/File:Flag_of_New_York_City.svg
https://en.wikipedia.org/wiki/File:Seal_of_New_York_City_BW.svg
https://en.wikipedia.org/wiki/File:NYC_Logo_Wolff_Olins.svg
https://en.wikipedia.org/wiki/File:Map_of_New_York_Highlighting_New_York_City.svg
https://en.wikipedia.org/wiki/File:Usa_edcp_location_map.svg
https://en.wikipedia.org/wiki/File:USA_New_York_location_map.svg
https://en.wikipedia.org/wiki/File:Map_of_New_York_Economic_Regions.svg
https://en.wikipedia.org/wiki/File:The_Purchase_of_Manhattan_Island.png
https://en.wikipedia.org/wiki/File:GezichtOpNieuwAmsterdam.jpg
https://en.wikipedia.org/wiki/File:A_view_of_Fort_George_with_the_city_of_New_York,_from_the_SW.jpg
https://en.wikipedia.org/wiki/File:BattleofLongisland.jpg
https://en.wikipedia.org/wiki/File:Hippolyte_Sebron_-_Rue_De_New-York_En_1840.jpg
https://en.wikipedia.org/wiki/File:Mulberry_Street_NYC_c1900_LOC_3g04637u_edit.jpg
https://en.

## Retrieving the scraped images to be generated as randomized collages for each city
Now that we have our images stored in Mongo lets create a function that pulls randomly draws pics for a collage

In [65]:
import random

def retrieve_pics(city, num_pics):
    '''
    randomly draws pictures src urls from the MongoDB
    inputs:
        city: name of city as a string
        num_pics: number of img urls to return 
    '''
    doc = collection.find_one({"city": city})
    num_images = len(doc['images'])
    #print(list(doc['images']))
    randomized_selection = random.sample(list(doc['images']),num_pics)
    
    return randomized_selection
       
retrieve_pics('Boston',10)   

['//upload.wikimedia.org/wikipedia/commons/5/58/Were_a_gay_and_happy_family_wagon.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/0/0b/2009_HanoverSt_Boston.jpg/800px-2009_HanoverSt_Boston.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/5/58/Seal_of_Suffolk_County%2C_Massachusetts.png/596px-Seal_of_Suffolk_County%2C_Massachusetts.png',
 '//upload.wikimedia.org/wikipedia/commons/thumb/0/09/US_Navy_090315-N-8110K-011_A_crowd_along_a_parade_route_in_South_Boston_cheers_Sailors_from_the_guided-missile_frigate_USS_Taylor_%28FFG_50%29_as_they_march_in_the_108th_Annual_St._Patrick%27s_Day_Parade.jpg/800px-thumbnail.jpg',
 '//upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Seal_of_Boston%2C_Massachusetts.svg/390px-Seal_of_Boston%2C_Massachusetts.svg.png',
 '//upload.wikimedia.org/wikipedia/commons/thumb/5/59/Haymarket_Square.JPG/800px-Haymarket_Square.JPG',
 '//upload.wikimedia.org/wikipedia/commons/thumb/a/a8/TremontSt_ca1843_Boston_byPhilipHarry_MFABoston.png/693px-TremontSt_

In [89]:
# Saving the MongoDB as a simple json file 

from bson.json_util import dumps
collection_json = dumps(collection.find())
with open('city_pics_urls.json',"w") as f:
    json.dump(collection_json,f)

In [13]:
# retrieving the custom made json file you made out of that junk mongoDB outputted *wink
import json
import random

def get_pic_urls(city, num_pics):
    """
    queries the pic urls json file and returns a random sampling of them
    
    No guarenttees for trying to get more than 6 pics
    """
    with open('city_pics_urls.JSON', 'r') as f:
        data = json.loads(f.read())
        city_urls = data[city]
        return random.sample(list(city_urls),num_pics)
    
get_pic_urls('New York', 10)

['https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/New_York_World%27s_Fair_August_1964.jpeg/800px-New_York_World%27s_Fair_August_1964.jpeg',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/Dag_Hammarskjold_outside_the_UN_building.jpg/491px-Dag_Hammarskjold_outside_the_UN_building.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Usa_edcp_location_map.svg/800px-Usa_edcp_location_map.svg.png',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/800px-Flag_of_New_York.svg.png',
 'https://upload.wikimedia.org/wikipedia/commons/f/f5/Terra.png',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/NYC_Hybrid_Taxi.JPG/800px-NYC_Hybrid_Taxi.JPG',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/1d/NYC_-_Washington_Square_Park_-_Arch.jpg/800px-NYC_-_Washington_Square_Park_-_Arch.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Empire_State_Building_%28HDR%29.jpg/405px-Empire_State_Building_%28HDR%2

In [162]:
import csv

with open('city_pics_urls.csv', 'r') as f:
    image_urls = []
    for row in csv.DictReader(f, fieldnames=['_id','city', 'images']):
        #get the row with the city name
        if row['city'] == "Seattle":
            #for some reason only the first url is actually under the images column
            first_image = row['images'].replace("[", '"')
            image_urls.append(first_image)
            if row[None]:
                print(row[None])
            
            

['//upload.wikimedia.org/wikipedia/commons/thumb/a/af/Aerial_Lake_Union_June_2012.jpg/800px-Aerial_Lake_Union_June_2012.jpg', '//upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Pike_Place_Market_1.jpg/800px-Pike_Place_Market_1.jpg', '//upload.wikimedia.org/wikipedia/commons/thumb/1/10/Seattle_Ferry.jpg/800px-Seattle_Ferry.jpg', '//upload.wikimedia.org/wikipedia/en/thumb/6/6d/Flag_of_Seattle.svg/262px-Flag_of_Seattle.svg.png', '//upload.wikimedia.org/wikipedia/en/0/04/City_of_Seattle%E2%80%94Seal.png', '//upload.wikimedia.org/wikipedia/commons/thumb/a/a4/King_County_Washington_Incorporated_and_Unincorporated_areas_Seattle_Highlighted.svg/800px-King_County_Washington_Incorporated_and_Unincorporated_areas_Seattle_Highlighted.svg.png', '//upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Usa_edcp_relief_location_map.png/800px-Usa_edcp_relief_location_map.png', '//upload.wikimedia.org/wikipedia/commons/thumb/e/e5/General_history%2C_Alaska_Yukon_Pacific_Exposition%2C_fully_illustrated_-_me

In [146]:
csv.reader?

Docstring:
csv_reader = reader(iterable [, dialect='excel']
                        [optional keyword args])
    for row in csv_reader:
        process(row)

The "iterable" argument can be any object that returns a line
of input for each iteration, such as a file object or a list.  The
optional "dialect" parameter is discussed below.  The function
also accepts optional keyword arguments which override settings
provided by the dialect.

The returned object is an iterator.  Each iteration returns a row
of the CSV file (which can span multiple input lines).
Type:      builtin_function_or_method
